In [2]:
import os 
import pandas as pd 
import tensorflow as tf 
import time 

from sklearn.model_selection import train_test_split

In [3]:
def convert_data(a, b):
    
    row_list =[]     

    for index, rows in a.iterrows(): 
        
        my_list = [rows[i] for i in a.columns]     
        row_list.append(my_list) 
    
    target = [rows for _, rows in b.items()]
        
    return row_list, target

In [18]:
address = "E:\\DKE related\\Studies\\Second Semester\\ATiML\\Codes\\Semester Project\\feature_vector.csv"

df = pd.read_csv(address)
df['genre'] = df['genre'].astype('category')
df['genre'] = df['genre'].cat.codes

X_train,X_test,y_train,y_test = train_test_split(df.iloc[:, 1:-1].to_numpy(), df.iloc[:, -1].to_numpy(),random_state=0)

df

,book_id,avg_sentence_length,avg_char_per_word,avg_punctuation_per_sentence,long_word_ratio,noun_ratio,verb_ratio,adverb_ratio,preposition_ratio,adjective_ratio,article_ratio,gerund_infinitive_ratio,downtoner_ratio,amplifier_ratio,demonstrative_ratio,type_token_ratio,genre
0,pg19459.epub,0.0445,4.5419,2.8576,0.2117,0.2211,0.1761,0.0599,0.1384,0.0854,0.0784,0.0211,0.0031,0.0033,0.0191,0.0070,5
1,pg12CarolGlassLook.epub,0.0735,4.0510,4.2273,0.1166,0.2020,0.2110,0.0785,0.1146,0.0565,0.0764,0.0249,0.0046,0.0050,0.0147,0.0127,5
2,pg9311.epub,0.0601,4.1475,3.9024,0.1327,0.2232,0.1958,0.0696,0.1156,0.0722,0.0621,0.0146,0.0032,0.0031,0.0164,0.0060,5
3,pg34849.epub,0.0375,4.4684,4.5543,0.1912,0.2367,0.1809,0.0529,0.1315,0.0721,0.0754,0.0194,0.0027,0.0030,0.0199,0.0114,5
4,pg17698.epub,0.0799,4.2311,2.8501,0.1571,0.2164,0.1939,0.0683,0.1246,0.0612,0.0738,0.0178,0.0047,0.0036,0.0175,0.0012,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,pg3828.epub,0.0812,4.2460,2.7203,0.1770,0.2253,0.1916,0.0482,0.1209,0.0676,0.0737,0.0177,0.0021,0.0016,0.0117,0.0039,4
90,pg15895.epub,0.0601,4.1501,3.2832,0.1323,0.2209,0.1893,0.0553,0.1191,0.0694,0.0808,0.0163,0.0025,0.0014,0.0123,0.0155,1
91,pg5662.epub,0.0792,3.9478,2.9638,0.0979,0.2053,0.2056,0.0640,0.0937,0.0675,0.0663,0.0136,0.0028,0.0041,0.0118,0.0824,2
92,pg19337DickensChristCarl.epub,0.0694,4.2876,3.5462,0.1669,0.2390,0.1855,0.0544,0.1252,0.0718,0.0764,0.0187,0.0015,0.0032,0.0164,0.0193,2


In [12]:
X_train

array([[5.2800e-02, 4.1324e+00, 3.0140e+00, ..., 2.6000e-03, 2.3900e-02,
        9.4000e-03],
       [6.3000e-02, 4.3200e+00, 3.3884e+00, ..., 1.3000e-03, 1.6000e-02,
        6.1000e-03],
       [8.2100e-02, 4.1984e+00, 3.2589e+00, ..., 5.0000e-03, 1.4400e-02,
        5.8000e-03],
       ...,
       [6.5800e-02, 4.1549e+00, 3.4849e+00, ..., 3.2000e-03, 2.2400e-02,
        2.4000e-03],
       [8.5000e-02, 3.9235e+00, 3.0112e+00, ..., 8.0000e-04, 2.0300e-02,
        1.7100e-02],
       [3.7600e-02, 4.3843e+00, 4.6118e+00, ..., 4.1000e-03, 2.3900e-02,
        2.4000e-03]])

In [13]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=500).batch(50).repeat(200)

test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_data = test_data.batch(64)

# for item in train_data:
#     print(item)

In [6]:
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))

model.fit(train_data, epochs=3)

result = model.evaluate(test_data)
print(model.metrics_names, result)

Train for 400 steps
Epoch 1/3
400/400 [==============================] - 1s 3ms/step - loss: 8.5917
Epoch 2/3
400/400 [==============================] - 1s 2ms/step - loss: 7.7417
Epoch 3/3
1/1 [==============================] - 0s 65ms/step - loss: 5.4625
['loss'] 5.4625420570373535


In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20)
# mse_test = model.evaluate(X_test, y_test)
# X_new = X_test[:3]
# y_pred = model.predict(X_new)


plt.plot(pd.DataFrame(history.history))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

y_pred

In [ ]:
df

In [ ]:
address = "E:\\DKE related\\Studies\\Second Semester\\ATiML\\Codes\\Semester Project\\feature_vector.csv"

df = pd.read_csv(address)
df['genre'] = df['genre'].astype('category')
df['genre'] = df['genre'].cat.codes

df

In [ ]:
X = df.iloc[:, 1:-1].to_numpy()
Y = df.iloc[:, -1].to_numpy()

scaler = StandardScaler()
X = scaler.fit_transform(X.astype(np.float64))



In [ ]:
X_train_val, X_test, Y_train_val, Y_test = train_test_split(df.iloc[:, 1:-1], df.iloc[:, -1], test_size=0.20, random_state = 0, shuffle=True, stratify = df.iloc[:, -1])

skf = StratifiedKFold(n_splits=5, shuffle=True)

for train_index, val_index in skf.split(X_train_val, Y_train_val):
    
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]

    print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)

    nb = MultinomialNB()

    %time nb.fit(X_train, Y_train)

    y_pred_nb = nb.predict(X_val)
    acc_nb = metrics.accuracy_score(y_val, y_pred_nb)
    print(acc_nb)
    


In [164]:
X = df.iloc[:, 1:-1].to_numpy()
Y = df.iloc[:, -1].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state = 0, shuffle=True, stratify = Y)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape


((70, 15), (24, 15), (70,), (24,))

In [168]:
nb = MultinomialNB()

%time nb.fit(X_train, Y_train)

Y_pred_nb = nb.predict(X_test)

acc_nb = metrics.accuracy_score(Y_test, Y_pred_nb)
print(acc_nb)

Wall time: 964 µs
0.375


In [ ]:
X

In [19]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn import metrics

val_acc_svm = 0
val_acc_logistic = 0
val_acc_nb = 0
val_acc_rf = 0

X = df.iloc[:, 1:-1].to_numpy()
Y = df.iloc[:, -1].to_numpy()

X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.20, random_state = 0, shuffle=True, stratify = Y)

skf = StratifiedKFold(n_splits=5, shuffle=True)

for train_index, val_index in skf.split(X_train_val, Y_train_val):
    
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    Y_train, Y_val = Y_train_val[train_index], Y_train_val[val_index]
    
    val_acc_svm += perform_SVM(X_train, Y_train, X_val, Y_val)
    val_acc_logistic += perform_Logistic(X_train, Y_train, X_val, Y_val)
    val_acc_nb += perform_NB(X_train, Y_train, X_val, Y_val)
    val_acc_rf += perform_random_forrest(X_train, Y_train, X_val, Y_val)

    # print("SVM: {}; Logistic: {}; NB: {}; RB: {}".format(round(acc_svm, 4), round(acc_logistic, 4), round(acc_nb, 4), round()))

val_acc_svm = float(val_acc_svm/5)
val_acc_logistic = float(val_acc_logistic/5)
val_acc_nb = float(val_acc_nb/5) 
val_acc_rf = float(val_acc_rf/5) 

print("Average Validation Accuracy")
print("SVM: {}; Logistic: {}; NB: {}; RB: {}".format(round(val_acc_svm, 4), round(val_acc_logistic, 4), round(val_acc_nb, 4), round(val_acc_rf, 4)))

# test accuracy 


fitting data to SVM
Wall time: 21 ms
fitting data to Logistic
Wall time: 57.8 ms
fitting data to Naive Bayes
Wall time: 1.01 ms
fitting data to SVM
Wall time: 14 ms
fitting data to Logistic
Wall time: 21 ms
fitting data to Naive Bayes
Wall time: 1.02 ms
fitting data to SVM
Wall time: 15 ms
fitting data to Logistic
Wall time: 20 ms
fitting data to Naive Bayes
Wall time: 0 ns
fitting data to SVM
Wall time: 13 ms
fitting data to Logistic
Wall time: 19 ms
fitting data to Naive Bayes
Wall time: 1.01 ms
fitting data to SVM
Wall time: 14 ms
fitting data to Logistic
Wall time: 19 ms
fitting data to Naive Bayes
Wall time: 999 µs
Average Validation Accuracy
SVM: 0.2267; Logistic: 0.24; NB: 0.2; RB: 0.3067


In [17]:
def perform_SVM(X_train, Y_train, X_val, Y_val):
    # SVM 
    
    svm_clf = Pipeline([        
        ("linear_svc", LinearSVC(C=1, loss="hinge")),
    ])

    print("fitting data to SVM")
    # svm_clf = SVC(gamma='auto')
    %time svm_clf.fit(X_train, Y_train)
    Y_pred_svm = svm_clf.predict(X_val)
    
    return metrics.accuracy_score(Y_val, Y_pred_svm)

def perform_Logistic(X_train, Y_train, X_val, Y_val):
    
    softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10)
    print("fitting data to Logistic")
    %time softmax_reg.fit(X_train, Y_train)
    Y_pred_logistic = softmax_reg.predict(X_val)    
    return metrics.accuracy_score(Y_val, Y_pred_logistic)

    # Multinomial Naive Bayes

def perform_NB(X_train, Y_train, X_val, Y_val):

    nb = MultinomialNB()
    print("fitting data to Naive Bayes")
    %time nb.fit(X_train, Y_train)
    Y_pred_nb = nb.predict(X_val)
    return metrics.accuracy_score(Y_val, Y_pred_nb)


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


def perform_random_forrest(X_train, Y_train, X_val, Y_val):

    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, Y_train)

    Y_pred = clf.predict(X_val)
    return metrics.accuracy_score(Y_val, Y_pred)